In [ ]:
import pandas as pd
from iminuit import Minuit 
from iminuit.cost import LeastSquares
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
from matplotlib.lines import Line2D

import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')



In [ ]:
def retta (x,a,b):
    return (a*x + b)

def sqrt(x, a, b):
    return np.sqrt(a*x + b)

In [ ]:
def read():
    Z1 = pd.read_csv('Analisi\Parte 1\Analisi Freq_Vb\F_Vb.CSV')
    Y = []
    S = []
    for i in range(len(Z1.columns.values)):
        if i != 0:
            z = Z1.iloc[:, i]
            y = float(np.mean(z))
            Y.append(y)
            s = float(np.std(z))
            S.append(s)
    return (Y, S)

In [ ]:
def ax_x (S, Y, E):
    X = [52.5, 53, 53.5, 53.8, 54, 54.3, 54.5, 54.8, 55, 55.3, 55.5, 55.8, 56]
    E1 = []
    X1 = []
    Y1 = []
    for i in range(len(X)):
        if Y[i] > S:
            X1.append(X[i])
            Y1.append(Y[i])
            E1.append(E[i])
    return X1, Y1, E1

In [ ]:
def graph(X, Y, S, a, b):
    fig, ax = plt.subplots(nrows = 1, ncols = 1)
    plt.plot(np.linspace(51, 56, 100000), sqrt(np.linspace(51, 56, 100000), a, b), label = 'Funzione di interpolazione')
    ax.set_xlabel("Threshold[mV]")
    ax.set_ylabel("Frequency[kHz]")
    ax.scatter(X, Y, color = 'black', label = 'Dati sperimentali')
    ax.errorbar(X, Y, S)
    ax.legend()
    plt.show()


In [ ]:
def analisi_sqrt(X, Y, S):

    L_S = LeastSquares(X, Y, S, sqrt)
    my_minuit = Minuit (L_S, a = 40, b = 1000)  
    my_minuit.migrad () 
    my_minuit.hesse ()   

#validità
    V = my_minuit.valid
    print('Validità: ', V)
    Q_squared = my_minuit.fval
    print ('Q-squared: ', Q_squared)
    N_dof = my_minuit.ndof
    print ('DOF: ', N_dof)
    p_v = 1 - chi2.cdf(Q_squared, N_dof)
    print('P value: ', p_v )

#valori dell'interpolazione
    a_f = my_minuit.values[0]
    b_f = my_minuit.values[1]

#visualizzazione 
    display(my_minuit)
    return (a_f, b_f)

def analisi_retta(X,Y,S):
    L_S = LeastSquares(X, Y, S, retta)
    my_minuit = Minuit (L_S, a = 40, b = 500)  
    my_minuit.migrad () 
    my_minuit.hesse ()   

#validità
    V = my_minuit.valid
    print('Validità: ', V)
    Q_squared = my_minuit.fval
    print ('Q-squared: ', Q_squared)
    N_dof = my_minuit.ndof
    print ('DOF: ', N_dof)
    p_v = 1 - chi2.cdf(Q_squared, N_dof)
    print('P value: ', p_v )

#valori dell'interpolazione
    a_f = my_minuit.values[0]
    b_f = my_minuit.values[1]

#visualizzazione 
    display(my_minuit)
    return (a_f, b_f, Q_squared/N_dof)

In [ ]:
Y = read()
X = ax_x(0.3, Y[0], Y[1])
Z = analisi_sqrt(X[0], X[1], X[2])
print('a = ', Z[0])
print('b = ', Z[1])

In [ ]:
print(Z[0], Z[1])
graph(X[0], X[1], X[2], Z[0], Z[1])

In [ ]:
X2 = ax_x(30, Y[0], Y[1])
Z2 = analisi_retta(X2[0], X2[1], X2[2])
print('a = ', Z2[0])
print('b = ', Z2[1])

In [ ]:
x = -Z2[1]/Z2[0]
print('X = ', x)

In [ ]:
def intercetta():
    G = [[], []]
    X = np.linspace(51, 52, 100000000)
    for i in range(len(X)):
        y = float(retta(X[i], Z2[0], Z2[1]))  # Use Z directly as a scalar
        if y <= 0.0000001:
             if  y >= -0.0000001:
                G[0].append(X[i]) 
                G[1].append(y)
    return G
#F = intercetta()
#print('Intercetta: ', F[0])  # Z is already a scalar

In [ ]:
X3 = analisi_retta(X2[0], np.sqrt(X2[1]), X2[2])

print('a = ', X3[0])
print('b = ', X3[1])

In [ ]:
x2 = -X3[1]/X3[0]
print('X2 = ', x2)

# Analisi V_br a seconda della soglia


In [ ]:
def read_mean(file_path):
    data = pd.read_excel(file_path)
    mean_values = np.mean(data.iloc[:, 1])
    std_values = np.std(data.iloc[:, 1])
    return mean_values, std_values

X = [52.75, 53, 53.25, 53.5, 53.75, 54, 54.25]

In [ ]:
Y1 = read_mean('Dati\Parte 1\Frequency_Vbias\TH3_VB52_75.xls')
Y2 = read_mean('Dati\Parte 1\Frequency_Vbias\TH3_VB53.xls')
Y3 = read_mean('Dati\Parte 1\Frequency_Vbias\TH3_VB53_25.xls')  
Y4 = read_mean('Dati\Parte 1\Frequency_Vbias\TH3VB35_5.xls')
Y5 = pd.read_csv('Dati\Parte 1\Frequency_Vbias\TH3_VB53_75.txt', sep = '\t')
Y5 = (Y5.iloc[:, 1].mean(), Y5.iloc[:, 1].std())
Y6 = read_mean('Dati\Parte 1\Frequency_Vbias\TH3_VB54.xls')
Y7 = read_mean('Dati\Parte 1\Frequency_Vbias\TH3_VB54_25.xls')

V1 = [Y1[0], Y2[0], Y3[0], Y4[0], Y5[0], Y6[0], Y7[0]]
S1 = [Y1[1], Y2[1], Y3[1], Y4[1], Y5[1], Y6[1], Y7[1]]

A1 = analisi_retta(X, V1, S1)
a1_err = 0.016
b1_err = 0.9
Zero1 = -A1[1]/A1[0]
print('Zero = ', Zero1, ' +/- ', Zero1 * np.sqrt((a1_err/A1[0])**2 + (b1_err/A1[1])**2))
print('a = ', A1[0], ' +/- ', a1_err)
print('b = ', A1[1], ' +/- ', b1_err)
print('Q_squared/N_dof = ', A1[2])


In [ ]:
K1 = read_mean('Dati\Parte 1\Frequency_Vbias\TH4_VB52_75.xls')
K2 = pd.read_csv('Dati\Parte 1\Frequency_Vbias\TH4_VB53.txt', sep = '\t')
K2 = (K2.iloc[:, 1].mean(), K2.iloc[:, 1].std())
K3 = read_mean('Dati\Parte 1\Frequency_Vbias\TH4_VB53_25.xls')
K4 = read_mean('Dati\Parte 1\Frequency_Vbias\TH4_VB53_5.xls')  
K5 = read_mean('Dati\Parte 1\Frequency_Vbias\TH4_VB53_75.xls')
K6 = read_mean('Dati\Parte 1\Frequency_Vbias\TH4_VB54.xls')
K7 = read_mean('Dati\Parte 1\Frequency_Vbias\TH4_VB54_25.xls')

V2_full = [K1[0], K2[0], K3[0], K4[0], K5[0], K6[0], K7[0]]
S2_full = [K1[1], K2[1], K3[1], K4[1], K5[1], K6[1], K7[1]]

X2 = [53, 53.25, 53.5, 53.75, 54, 54.25]
V2 = [ K2[0], K3[0], K4[0], K5[0], K6[0], K7[0]]
S2 = [ K2[1], K3[1], K4[1], K5[1], K6[1], K7[1]]

A2 = analisi_retta(X2, V2, S2)
a2_err = 0.012
b2_err = 0.6
Zero2 = -A2[1]/A2[0]
print('Zero = ', Zero2, ' +/- ', Zero2 * np.sqrt((a2_err/A2[0])**2 + (b2_err/A2[1])**2))
print('a = ', A2[0], ' +/- ', a2_err)
print('b = ', A2[1], ' +/- ', b2_err)
print('Q_squared/N_dof = ', A2[2])

In [ ]:
Z1 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB52_75.xls')
Z2 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB53.xls')
Z3 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB53_25.xls')  
Z4 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB53_5.xls')
Z5 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB53_75.xls')
Z6 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB54.xls')
Z7 = read_mean('Dati\Parte 1\Frequency_Vbias\TH5_VB54_25.xls')


V3 = [Z1[0], Z2[0], Z3[0], Z4[0], Z5[0], Z6[0], Z7[0]]
S3 = [Z1[1], Z2[1], Z3[1], Z4[1], Z5[1], Z6[1], Z7[1]]

X_redox = [53, 53.25, 53.5, 53.75, 54, 54.25]
V3_redox = [Z2[0], Z3[0], Z4[0], Z5[0], Z6[0], Z7[0]]
S3_redox = [Z2[1], Z3[1], Z4[1], Z5[1], Z6[1], Z7[1]]   

A3 = analisi_retta(X_redox, V3_redox, S3_redox)
a3_err = 0.021
b3_err = 1.1
Zero3 = -A3[1]/A3[0]
print('Zero = ', Zero3, ' +/- ', Zero3 * np.sqrt((a3_err/A3[0])**2 + (b3_err/A3[1])**2))
print('a = ', A3[0], ' +/- ', a3_err)
print('b = ', A3[1], ' +/- ', b3_err)
print('Q_squared/N_dof = ', A3[2])

In [ ]:
assex = np.linspace(52.75, 54.25, 100000)
assey1 = retta(assex, A1[0], A1[1]) 
assey2 = retta(assex, A2[0], A2[1])
assey3 = retta(assex, A3[0], A3[1])
fig, ax = plt.subplots(nrows = 1, ncols = 1)
plt.Figure(figsize=(10, 6))
plt.plot(assex, assey1, color = 'red', label = 'Retta interpolata per TH = 3mV')
ax.scatter(X, V1, color = 'blue', label = 'Dati sperimentali TH = 3mV', s = 5)
ax.errorbar(X, V1, S1, fmt = 'o', color = 'blue')
ax.grid()
ax.set_xlabel("V_Bias[V]")
ax.set_ylabel("Frequenza[kHz]")
ax.legend()
plt.savefig('Analisi\Parte 1\Analisi Freq_Vb\TH3.png', dpi = 300)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
fig, ax = plt.subplots(nrows = 1, ncols = 1)
plt.plot(assex, assey2, color  = 'red', label = 'Retta interpolata per TH = 4mV')
ax.scatter(X, V2_full, color = 'blue', label = 'Dati sperimentali TH = 4mV', s = 5)
ax.errorbar(X, V2_full, S2_full, fmt = 'o', color = 'blue')
ax.grid()
ax.set_xlabel("V_Bias[V]")
ax.set_ylabel("Frequenza[kHz]")
ax.legend(handles=)
plt.savefig('Analisi\Parte 1\Analisi Freq_Vb\TH4.png', dpi = 300)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
fig, ax = plt.subplots(nrows = 1, ncols = 1)
plt.plot(assex, assey3, color = 'red', label = 'Retta interpolata per TH = 5mV')
ax.scatter(X, V3, color = 'blue', label = 'Dati sperimentali TH = 5mV', s = 5)
ax.errorbar(X, V3, S3, fmt = 'o', color = 'blue')
ax.grid()
ax.set_xlabel("V_Bias[V]")
ax.set_ylabel("Frequenza[kHz]")
ax.legend()
plt.savefig('Analisi\Parte 1\Analisi Freq_Vb\TH5.png', dpi = 300)
plt.show()


In [ ]:
assex = np.linspace(52.75, 54.25, 100000)
assey1 = retta(assex, A1[0], A1[1]) 
assey2 = retta(assex, A2[0], A2[1])
assey3 = retta(assex, A3[0], A3[1])
fig, ax = plt.subplots(nrows = 1, ncols = 1)
plt.figure(figsize=(10, 6))

legend_elements = [
    Line2D([0], [0], color='red', lw=2, label = 'Fit, $\tilde\chi^2$ =' {c2r_B_p_C:.3g})

plt.plot(assex, assey1, color = 'red', label = 'Retta interpolata per TH = 3mV')
ax.scatter(X, V1, color = 'blue', label = 'Dati sperimentali TH = 3mV', s = 5)
ax.errorbar(X, V1, S1, fmt = 'o', color = 'blue')
ax.grid()
ax.set_xlabel("V_Bias[V]")
ax.set_ylabel("Frequenza[kHz]")
ax.legend()
plt.savefig('Analisi\Parte 1\Analisi Freq_Vb\TH3.png', dpi = 300)
plt.show()